# A Simple Fugu Example

This quick example will run through how to use a Fugu in a most basic way.  The objective here is to take a vector input, apply two separate dot products to this input, and then determine if the resulting values are above 1.0.

We begin with some basic imports.  In the future, we should probably better organize fugu so that we have the brick in a separate directory, e.g. fugu.bricks.

In [3]:
import networkx as nx
import numpy as np
from fugu.bricks import Vector_Input, Dot, Copy, Threshold
from fugu import Scaffold

NetworkX actually doesn't need to be imported here, but you'll quickly run into issues if you start playing around the with objects without it.

Here's a quick run-down of the fugu objects we use here:
- `Scaffold`:  The scaffold object is a container/graph object that organizes and holds computational nodes called 'bricks'.  Various types of bricks can be combined to construct a more sophisticated algorithm.  `Scaffold` is the main object for end-users.  The general workflow is something like this:
    1. Define a `Scaffold` object
    2. Attach bricks to the `Scaffold`, linking them together
    3. 'Build' the computational graph by calling `Scaffold.lay_bricks`
    4. Call `Scaffold.evaluate` and perform any post-processing to your answer.
A few todo-type caveats:  
    1. The `Scaffold` object should, itself, be a `Brick`.  This is critical but unfortunately currently missing.
    2. Inputs that take both pre-computed values and input spikes will not lay correctly.
    3. Nodes only accept input from the first output of an input node.
    4. The code is missing a large portion of expected 'checks' so right now it is on the user to determine if they are doing something allowed or disallowed.
    5. Brad kindly provided some code to determine how to add delays so that you can synchronize portions of the Scaffold.  I, rudely, broke this code.  Specifically, I believe the `Scaffold.resolve_timings` function should still work, but the underlying brick will not build correctly.

- `Spike_Input`: A `Brick` that provides spike inputs in the form of vectors.  For a single input, a numpy array (as you see below) is sufficient.  For inputs at various timesteps, use the flag `time_dimension`.  For generic vector input, use `coding=Raster`, though the user can specify an output coding as desired.
- `Dot`: A `Brick` that provides a dot product of two vectors.  While this could be updated in the future, right now the only implementation is uses weights, and the analog value is the output of the brick. This is referred to as `current` output coding (to signify threshold operation).  You can then link this with other bricks that accept `current` to produce spikes.  Understand that we cannot do very much with a signal when it is in `current` coding.  This responsiblilty is left to the programmer.  Easy things to do would be, for example, add or multiply (we'd be able to write these bricks).
- `Copy`: Another `Brick`.  Provides *n* (default 2) copies of the input signal in the same shape.  Each output is given its own output channel in the scaffold (see below).  We may be able to remove this brick in the future, but for now it is useful to prevent multiple bricks for adjusting the output of an upstream brick.
- `Threshold`:  Another `Brick`.  Provides a basic thresholding.  Useful, for example, to perform a threshold which converts `current` coding into spikes. 

In [4]:
scaffold = Scaffold()
scaffold.add_brick(Spike_Input(np.array([1,0,1]), coding='Raster'), 'input' )
scaffold.add_brick(Copy())

NameError: name 'Spike_Input' is not defined

When we add a `Brick` to a `Scaffold`, by default, it adds it to the last node on the graph (generally, the last brick added).  Most bricks provide one ouput channel; separate output channels allow for a brick to send separate output signals to separate bricks.  In this case, `Copy` provides two output channels coming from the same node (node number 1).  Below, we connect a different `Dot` brick to each of these output channels.  By default, a brick connects to the first output channel of a node if none is specified.  We also name them for fun!

In [ ]:
scaffold.add_brick(Dot([1,0,1], name='ADotOperator'), (1,0))  #Connects this Dot brick to node 1, channel 0 
scaffold.add_brick(Dot([0,0,1], name='AnotherDotOperator'), (1,1)) #Connects this Dot brick to node 1, channel 1

And now, we apply thresholds connecting to the `Dot` bricks.  Conveniently, we've named the bricks, so we can easily determine what node numbers they have...

In [5]:
[scaffold.circuit.nodes[node]['brick'].name for node in scaffold.circuit.nodes]

[]

In [6]:
scaffold.add_brick(Threshold(1.75, name='Neuron 13'),(2,0), output=True)
scaffold.add_brick(Threshold(1.25, name='Neuron 14'),(3,0), output=True)
graph = scaffold.lay_bricks()

KeyError: 'name'

`Scaffold.lay_bricks` performs all the necessary steps to ready a full graph of neurons (not just computational nodes) for a hardware platform or simulator.  You can preview the graph if you want:

In [7]:
[scaffold.graph.nodes[node] for node in scaffold.graph.nodes]

AttributeError: 'NoneType' object has no attribute 'nodes'

A little description of what is going on here:  'threshold', 'decay', and 'p' are all neuron parameters used by the the evaluation platform (e.g. the simulator).  'index' is a tuple describing the local index of that neuron relative to its encoding scheme.  So, for example, if you have a two-dimensional, three-bit binary encoding, your shape might be (3,2) and so an index of (1,2) represents the 1st bit (counting from the 0th bit) of the 2nd number.  

We can now evaluate the computation.  Right now, we only support the ds simulator. 
https://gitlab.sandia.gov/nerl/ds_simulator/tree/master 
Everyone should have access to it.  Just copy ds.py into your working directory. 

In [ ]:
result = scaffold.evaluate(backend='ds')
print(result)

For now, `Scaffold.evaluate` returns a spike history.  So, the fact that neuron 13 fired on timestep 2 signifies that $[1,0,1]*[1,0,1]^T > 1.75 $.  In the future, we should have output layers capable of decoding the spike results.

That's it for this example.  Hopefully, there will be a walkthrough of the code soon.  A lot of the code is boilerplate once you understand how data is being moved back and forth.  We have a lot of thinking to do on what design we should have.